In [1]:
print(1)

1


# 第一问

In [3]:
import numpy as np

def solve_problem_1():
    """
    解决数学建模竞赛的问题1。
    在给定的一系列参数下，计算单枚烟幕弹对导弹M1的有效遮蔽时长。
    """

    # --- 1. 常量与初始条件 ---

    # 物理常量
    G = 9.8  # 重力加速度 (m/s^2)

    # 题目给出的初始位置 (单位: 米)
    P_M1_INITIAL = np.array([20000.0, 0.0, 2000.0])   # 导弹M1
    P_FY1_INITIAL = np.array([17800.0, 0.0, 1800.0])  # 无人机FY1

    # 目标位置 (单位: 米)
    P_FAKE_TARGET = np.array([0.0, 0.0, 0.0]) # 假目标
    # 为简化计算，将圆柱形真目标近似为其中心点进行视线判断
    P_TRUE_TARGET_CENTER = np.array([0.0, 200.0, 5.0]) 

    # 题目给出的速度 (单位: m/s)
    V_M1_SPEED = 300.0
    V_FY1_SPEED = 120.0

    # 问题1给定的时间参数 (单位: 秒)
    T_DROP = 1.5                # 投放时间
    T_EXPLODE_AFTER_DROP = 3.6  # 投放后到引爆的时间

    # 烟幕云团属性
    CLOUD_RADIUS = 10.0               # 云团半径 (米)
    CLOUD_SINK_SPEED = 3.0            # 云团下沉速度 (m/s)
    CLOUD_EFFECTIVE_DURATION = 20.0   # 有效遮蔽持续时间 (秒)

    # --- 2. 关键事件与向量的预计算 ---

    # FY1保持高度不变，朝向假目标（原点）飞行，因此速度向量为x轴负方向
    V_FY1 = np.array([-V_FY1_SPEED, 0.0, 0.0])

    # 计算烟幕弹的投放位置
    P_DROP = P_FY1_INITIAL + V_FY1 * T_DROP

    # 计算引爆发生的时刻
    T_DETONATION = T_DROP + T_EXPLODE_AFTER_DROP

    # 计算引爆位置（烟幕弹投放后做抛物线运动）
    # 烟幕弹的初速度等于投放时无人机的速度
    x_det = P_DROP[0] + V_FY1[0] * T_EXPLODE_AFTER_DROP
    y_det = P_DROP[1]  # y方向无运动
    z_det = P_DROP[2] - 0.5 * G * T_EXPLODE_AFTER_DROP**2
    P_DETONATION = np.array([x_det, y_det, z_det])

    # 计算导弹M1的恒定速度向量
    direction_m1 = P_FAKE_TARGET - P_M1_INITIAL
    V_M1 = (V_M1_SPEED / np.linalg.norm(direction_m1)) * direction_m1


    # --- 3. 动态位置与遮蔽计算函数 ---

    def get_missile_pos(t):
        """计算t时刻导弹M1的位置。"""
        return P_M1_INITIAL + V_M1 * t

    def get_cloud_center_pos(t):
        """计算t时刻烟幕云团中心的位置。"""
        # 云团仅在引爆后存在并开始下沉
        time_since_detonation = t - T_DETONATION
        sink_distance = CLOUD_SINK_SPEED * time_since_detonation
        return P_DETONATION - np.array([0.0, 0.0, sink_distance])

    def calculate_shielding_distance(t):
        """
        计算在t时刻，云团中心到导弹-真目标视线（三维空间直线）的距离。
        """
        missile_pos = get_missile_pos(t)
        cloud_pos = get_cloud_center_pos(t)
        
        # 三维空间中点到直线距离公式所需的向量
        vec_missile_to_cloud = cloud_pos - missile_pos
        vec_missile_to_target = P_TRUE_TARGET_CENTER - missile_pos
        
        # 距离公式: d = || 向量1 x 向量2 || / || 向量2 ||
        cross_product_magnitude = np.linalg.norm(np.cross(vec_missile_to_cloud, vec_missile_to_target))
        line_vector_magnitude = np.linalg.norm(vec_missile_to_target)
        
        if line_vector_magnitude == 0: return 0.0
        
        distance = cross_product_magnitude / line_vector_magnitude
        return distance


    # --- 4. 通过仿真循环求解遮蔽时长 ---

    time_step = 0.001  # 使用微小时间步长以保证精度
    start_time = T_DETONATION
    end_time = T_DETONATION + CLOUD_EFFECTIVE_DURATION
    
    total_shielding_time = 0.0
    shield_start_time = -1
    
    was_shielded_previously = False # 用于检测遮蔽状态变化的标志位

    # 遍历烟幕的整个有效时间窗口
    for t in np.arange(start_time, end_time, time_step):
        distance = calculate_shielding_distance(t)
        is_shielded_now = (distance <= CLOUD_RADIUS)
        
        if is_shielded_now:
            total_shielding_time += time_step
        
        # 检测遮蔽区间的精确开始和结束时间
        if is_shielded_now and not was_shielded_previously:
            shield_start_time = t
        elif not is_shielded_now and was_shielded_previously:
            shield_end_time = t

        was_shielded_previously = is_shielded_now

    # --- 5. 输出结果 ---
    
    print("--- 求解结果：问题1 ---")
    print(f"\n关键事件参数:")
    print(f"  烟幕弹投放时间: {T_DROP:.3f} s")
    print(f"  烟幕弹引爆时间: {T_DETONATION:.3f} s")
    print(f"  烟幕弹引爆点坐标: ({P_DETONATION[0]:.3f}, {P_DETONATION[1]:.3f}, {P_DETONATION[2]:.3f})")

    print(f"\n遮蔽效果分析:")
    if shield_start_time != -1:
        print(f"  有效遮蔽开始于: t ≈ {shield_start_time:.3f} s")
        print(f"  有效遮蔽结束于: t ≈ {shield_end_time:.3f} s")
        print(f"\n  最终结果：有效遮蔽总时长 ≈ {total_shielding_time:.3f} s")
    else:
        print("  在给定条件下，未能形成有效遮蔽。")

# 运行求解函数
solve_problem_1()

--- 求解结果：问题1 ---

关键事件参数:
  烟幕弹投放时间: 1.500 s
  烟幕弹引爆时间: 5.100 s
  烟幕弹引爆点坐标: (17188.000, 0.000, 1736.496)

遮蔽效果分析:
  有效遮蔽开始于: t ≈ 8.014 s
  有效遮蔽结束于: t ≈ 12.051 s

  最终结果：有效遮蔽总时长 ≈ 4.037 s
